In [1]:
import pandas as pd
from pycaret.regression import *

df = pd.read_csv("Dataset/rumah123_update_tipe.csv")

df = df.drop(columns=["Unnamed: 0", "name"], errors='ignore')
df


,price,location,bedroom,bathroom,land_area,building_area
0,1680.0,rungkut,3,3,67.0,120.0
1,950.0,medokan ayu,3,3,72.0,140.0
2,500.0,wonorejo,2,1,30.0,36.0
3,950.0,sukolilo,2,2,50.0,55.0
4,600.0,kebraon,2,1,50.0,56.0
...,...,...,...,...,...,...
25929,1600.0,gununganyar,3,1,144.0,70.0
25930,1700.0,mulyorejo,5,2,120.0,200.0
25931,1300.0,wonorejo,5,3,120.0,180.0
25932,225.0,lainnya,6,4,642.0,600.0


In [2]:
df.duplicated().sum()

9650

In [3]:
df = df.drop_duplicates()

In [4]:
df.duplicated().sum()

0

In [ ]:
# import ydata_profiling as pp

# profile = pp.ProfileReport(df)
# profile.to_file('Report_Rumah123.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 90.31it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
setup(
    df, 
    target='price',
    transform_target=True, 
    normalize=True, 
    normalize_method='minmax',
    remove_outliers=True, 
    outliers_threshold=0.05,
    categorical_features=['location'],
    session_id=42
)

,Description,Value
0,Session id,42
1,Target,price
2,Target type,Regression
3,Original data shape,"(16284, 6)"
4,Transformed data shape,"(15714, 6)"
5,Transformed train set shape,"(10828, 6)"
6,Transformed test set shape,"(4886, 6)"
7,Numeric features,4
8,Categorical features,1
9,Preprocess,True


In [7]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:40:34
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,216.0381,80796.2054,284.1377,0.6448,0.2671,0.2147,8.4290
lightgbm,Light Gradient Boosting Machine,216.4554,80909.0006,284.3328,0.6443,0.2686,0.2168,0.2720
xgboost,Extreme Gradient Boosting,212.8403,81024.0578,284.5880,0.6438,0.2671,0.2101,0.1020
rf,Random Forest Regressor,211.7715,81841.9859,285.9653,0.6402,0.2674,0.2093,1.3350
et,Extra Trees Regressor,214.5166,85927.5674,292.9872,0.6223,0.2738,0.2116,0.3310
gbr,Gradient Boosting Regressor,227.5186,87540.9364,295.7831,0.6152,0.2787,0.2283,0.1450
knn,K Neighbors Regressor,235.2453,98234.4891,313.2836,0.5682,0.2921,0.2348,0.7750
ada,AdaBoost Regressor,274.2405,110864.2467,332.8884,0.5129,0.3191,0.2817,0.0960
dt,Decision Tree Regressor,260.1409,137754.5176,371.0493,0.3946,0.3451,0.2506,0.7410
omp,Orthogonal Matching Pursuit,344.1930,175125.9219,418.3441,0.2309,0.4037,0.3748,0.0940


In [12]:
tuned_model = tune_model(best, search_library='scikit-optimize', search_algorithm='bayesian')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,211.9303,78444.1690,280.0789,0.6689,0.2629,0.2099
1,209.4550,76686.5136,276.9233,0.6522,0.2587,0.2061
2,223.0196,81744.7398,285.9104,0.6424,0.2691,0.2220
3,213.7312,80723.4612,284.1187,0.6476,0.2642,0.2103
4,221.3870,81737.2657,285.8973,0.6463,0.2730,0.2226
5,215.5313,76821.7738,277.1674,0.6660,0.2556,0.2091
6,203.2342,69880.0503,264.3484,0.6908,0.2499,0.1988
7,217.6905,85820.6424,292.9516,0.6125,0.2759,0.2198
8,219.2525,85312.8259,292.0836,0.6298,0.2834,0.2307


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [13]:
holdout_pred = predict_model(tuned_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,216.3680,81141.0939,284.8528,0.6360,0.2652,0.2128


In [14]:
new_data = df.copy()
new_data.drop('price', axis=1, inplace=True)
new_data.head()

,location,bedroom,bathroom,land_area,building_area
0,rungkut,3,3,67.0,120.0
1,medokan ayu,3,3,72.0,140.0
2,wonorejo,2,1,30.0,36.0
3,sukolilo,2,2,50.0,55.0
4,kebraon,2,1,50.0,56.0


In [15]:
predictions = predict_model(tuned_model, data = new_data)
predictions.head()

,location,bedroom,bathroom,land_area,building_area,prediction_label
0,rungkut,3,3,67.0,120.0,1329.560463
1,medokan ayu,3,3,72.0,140.0,1210.658331
2,wonorejo,2,1,30.0,36.0,402.683862
3,sukolilo,2,2,50.0,55.0,906.907631
4,kebraon,2,1,50.0,56.0,678.085467


In [16]:
save_model(tuned_model, 'house-price-rf')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('target_transformation',
                  TransformerWrapperWithInverse(transformer=TargetTransformer(estimator=PowerTransformer(standardize=False)))),
                 ('numerical_imputer',
                  TransformerWrapper(include=['bedroom', 'bathroom', 'land_area',
                                              'building_area'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(...
                 ('rest_encoding',
                  TransformerWrapper(include=['location'],
                                     transformer=TargetEncoder(cols=['location'],
                                                               handle_missing='return_nan'))),
                 ('remove_outliers',
                  TransformerWrapper(transformer=RemoveOutliers(random_state=42))),
                 ('normalize', TransformerWrapper(transformer=